<h1 align="center"><font color='red'>Web Scrapping</font></h1>

In [1]:
#Importing important libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

**Note:**

Web scrapping from website: https://www.opentable.ie/r/sophies-at-the-dean-galway

Check the resturant location from your location = '<a href="https://www.google.com/maps/dir/53.2952566,-9.0605153/Sophie%27s+Rooftop+Galway,+78+Prospect+Hill,+Galway,+H91+WR6P/@53.2880857,-9.0948586,13z/data=!3m1!4b1!4m9!4m8!1m1!4e1!1m5!1m1!1s0x485b97312ec7ef3f:0x529d2ae04bf09f24!2m2!1d-9.0469633!2d53.2769277?entry=ttu" target="_blank">Sophie\'s Rooftop Galway Location</a>'



**Description:**
The Python script titled "Scraping OpenTable Reviews for Sophie's at The Dean, Galway" is designed to extract reviews from the OpenTable platform for the specified restaurant. The script utilizes custom headers, particularly a User-Agent string, to mimic browser behavior. By employing the requests library, the script sends HTTP GET requests to a series of URLs, which are previously generated to cover multiple pages of reviews. Upon receiving responses, the script utilizes BeautifulSoup to parse the HTML content. It then searches for specific <div> elements with the class attribute 'MpiILQAMSSg-', which typically contain individual review data. For each found item, representing a review, it appends the corresponding HTML snippet to a list named items. This script essentially automates the process of fetching and parsing reviews, enabling further analysis or processing of the extracted data.

In [2]:
#URLs for OpenTable Reviews of Sophie's at The Dean, Galway
url_1=[]

#Looping to get data one by one                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        till page 16
for i in range(1,16):
    url= "https://www.opentable.ie/r/sophies-at-the-dean-galway?page={}&sortBy=newestReview".format(i)
    url_1.append(url) 

In [3]:
#Scraping OpenTable Reviews for Sophie's at The Dean, Galway with Custom Headers
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

items=[]

for rev in url_1:
    search_response=requests.get(rev,headers=headers)
    search_response.text
    soup =BeautifulSoup(search_response.text,"lxml")
    for item in soup.find_all('div', attrs = {'class': 'MpiILQAMSSg-'}):
        items.append(item) 

In [4]:
#fetching reviews using class from html
review_2=[]

for i in items: 
    review=i.find('span',{'class': 'l9bbXUdC9v0- ZatlKKd1hyc- ukvN6yaH1Ds-'}).text.strip()
    review_2.append(review)

In [5]:
#Creating dataframe for fecthed reveiws
df_review = pd.DataFrame({'Reviews': review_2})
df_review.head()

,Reviews
0,Love it. Had wings downstairs in elephant and ...
1,Beautiful ambience. Nice service stuff. Took c...
2,"Very accomodating of our party, including our ..."
3,Very good service and good food. Restaurant wa...
4,"We booked a table for dinner, to celebrate a f..."


In [6]:
#Checking the length of reviews
len(df_review)

360

In [7]:
#fetching rating of food using class from html
rating_2 = []

for item in items:
    # Find all 'li' tags for each category rating within the item
    li_tags = item.find_all('li', {'class': '-k5xpTfSXac-'})
    for li in li_tags:
        category_rating = li.text.strip().split(" ")
        category = category_rating[0]=='Food'  # Category name, e.g., "Food"
        rating = category_rating[-1]  # Rating value
        rating_2.append((category, rating))
        true_ratings = [rating for is_true, rating in rating_2 if is_true]

# Convert list to DataFrame
df_rating = pd.DataFrame({'Ratings': true_ratings})

# Print DataFrame
print(df_rating.head())

  Ratings
0       5
1       5
2       4
3       4
4       3


In [8]:
# Convert 'Ratings' column to numeric
df_rating['Ratings'] = pd.to_numeric(df_rating['Ratings'])

# Define a function to map numerical ratings to sentiment categories
def map_sentiment(rating):
    if rating <=2:
        return 0  # bad
    elif rating == 3:
        return 1  # neutral
    elif rating in [4, 5]:
        return 2  # positive
    else:
        return -1  # unknown

# Apply the mapping function to the 'Ratings' column and remove it
df_rating['Sentiment'] = df_rating['Ratings'].apply(map_sentiment)
df_rating.drop(columns=['Ratings'], inplace=True)

# Print the modified DataFrame
print(df_rating.head())

   Sentiment
0          2
1          2
2          2
3          2
4          1


In [9]:
len(df_rating)

360

In [10]:
# Merge the two DataFrames
df = pd.concat([df_review, df_rating], axis=1)

# Print the merged DataFrame
print(df)

                                               Reviews  Sentiment
0    Love it. Had wings downstairs in elephant and ...          2
1    Beautiful ambience. Nice service stuff. Took c...          2
2    Very accomodating of our party, including our ...          2
3    Very good service and good food. Restaurant wa...          2
4    We booked a table for dinner, to celebrate a f...          1
..                                                 ...        ...
355  Great cocktails and atmosphere, will definitel...          2
356  Fabulous experience \nDelicious meal\nExcellen...          2
357  Food and cocktails were phenomenal, will be ba...          2
358  Few teething problems, really think the place ...          1
359  Loved everything about Sophieâs. Will 100% b...          2

[360 rows x 2 columns]


In [11]:
#Saving to csv file 
df.to_csv('fetched_resturant_file.csv',index=False)

**Take away:**

Through the process of web scraping, we've successfully acquired reviews and corresponding sentiment ratings, representing the rating of food. This data will serve as a pivotal component for our Assignment 3, where we'll utilize it for training various machine learning models. Subsequently, these trained models will be leveraged to predict the sentiment of new reviews provided by customers. By incorporating this scraped data into our assignment, we aim to develop robust predictive models capable of discerning sentiment with high accuracy, thereby enhancing our understanding of customer feedback and satisfaction levels.

<p style="text-align:center; color:#ff5733; font-size:24px; font-weight:bold; font-style:italic;">------Thanks for now------</p>
